In [2]:
!pip install tensorflow
# !pip install -q tensorflow-model-optimization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.6/479.6 MB 16.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 16.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 18.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 19.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 20.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.26.1
    Uninstalling protobuf-5.26.1:
      Successfully uninstalled protobuf-5.26.1
  Attempting uninstall: numpy
    Found exist

In [3]:
import tensorflow as tf
from tensorflow_model_optimization.python.core.keras.compat import keras

import numpy as np
import tempfile
import zipfile
import os

from model_profiler import model_profiler

2025-04-03 11:32:36.916365: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-03 11:32:36.954243: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-03 11:32:36.954971: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-03 11:32:37.614894: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images  = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation=tf.nn.relu),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_images,
    train_labels,
    validation_split=0.1,
    epochs=10
)

11490434/11490434 [==============================] - 1s 0us/step
Epoch 1/10


2025-04-03 11:32:48.944791: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-03 11:32:49.159229: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


1688/1688 [==============================] - 12s 7ms/step - loss: 0.2882 - accuracy: 0.9183 - val_loss: 0.1256 - val_accuracy: 0.9635
Epoch 2/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1182 - accuracy: 0.9662 - val_loss: 0.0850 - val_accuracy: 0.9767
Epoch 3/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0871 - accuracy: 0.9740 - val_loss: 0.0702 - val_accuracy: 0.9818
Epoch 4/10
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0714 - accuracy: 0.9789 - val_loss: 0.0665 - val_accuracy: 0.9830
Epoch 5/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.0614 - accuracy: 0.9813 - val_loss: 0.0593 - val_accuracy: 0.9845
Epoch 6/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0540 - accuracy: 0.9839 - val_loss: 0.0631 - val_accuracy: 0.9842
Epoch 7/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0487 - accuracy: 0.9848 - val_loss: 0.0583 - val_accuracy: 0.9

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 12)        120       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 12)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 2028)              0         
                                                                 
 dense (Dense)               (None, 10)                20290     
                                                                 
Total params: 20410 (79.73 KB)
Trainable params: 20410 (79.73 KB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [6]:
print(model_profiler(model, 128))

| Model Profile                    | Value         | Unit    |
|----------------------------------|---------------|---------|
| Selected GPUs                    | None Detected | GPU IDs |
| No. of FLOPs                     | 0.0           | BFLOPs  |
| GPU Memory Requirement           | 0.0062        | GB      |
| Model Parameters                 | 0.0204        | Million |
| Memory Required by Model Weights | 0.0779        | MB      |


2025-04-03 11:35:10.583619: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-03 11:35:10.589089: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

keras_file = '/src/init_model.h5'
print('Saving model to: ', keras_file)
keras.models.save_model(model, keras_file, include_optimizer=False)

Baseline test accuracy: 0.9818000197410583
Saving model to:  /src/init_model.h5


/tmp/ipykernel_40/2115648425.py:8: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  keras.models.save_model(model, keras_file, include_optimizer=False)


In [8]:
for w in model.weights:
    print(w.name, w.numpy())

conv2d/kernel:0 [[[[ 0.64077187  0.43250504 -0.83814985  0.07134694  0.0628445
    -0.46368307  0.37904808  0.8151185  -0.04636772  0.278506
    -1.09907     0.15333119]]

  [[ 0.9119164   0.2704301  -0.19328047 -0.70747447  0.23428154
     0.3691828   0.34915763  0.5253925   0.31428316  0.3000083
    -0.84211934  0.1376472 ]]

  [[ 0.92826295 -1.0867482   0.08003949 -0.9335634   0.37558675
     0.42365554 -0.07505465 -1.0386263  -0.77900094 -0.44590425
    -0.14758453 -0.09695109]]]


 [[[-0.29868412  0.7922094  -0.5168316   0.7944505   0.07161665
     0.38733372 -0.08406575  1.0821877  -0.13897736 -0.30250108
    -0.6132253   0.3301399 ]]

  [[ 0.39916244  0.19270536  0.01370717  0.05316766  0.14108358
     0.40726933 -0.141179    0.17478572 -0.5516148   0.42308643
     0.6352827   0.49112305]]

  [[ 0.5651289  -1.1630152  -0.02701709 -0.20516421 -0.27427202
    -0.35407698 -0.19408044 -1.2392026  -0.28797403  0.08683338
     0.86733365  0.19957688]]]


 [[[-1.4549811   0.74576855  0

In [9]:
import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 8,
  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS
}

# Cluster a whole model
clustered_model = cluster_weights(model, **clustering_params)

# Use smaller learning rate for fine-tuning clustered model
opt = keras.optimizers.Adam(learning_rate=1e-5)

clustered_model.compile(
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

clustered_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cluster_reshape (ClusterWe  (None, 28, 28, 1)         0         
 ights)                                                          
                                                                 
 cluster_conv2d (ClusterWei  (None, 26, 26, 12)        236       
 ghts)                                                           
                                                                 
 cluster_max_pooling2d (Clu  (None, 13, 13, 12)        0         
 sterWeights)                                                    
                                                                 
 cluster_flatten (ClusterWe  (None, 2028)              0         
 ights)                                                          
                                                                 
 cluster_dense (ClusterWeig  (None, 10)                4

In [10]:
print(model_profiler(clustered_model, 128))

| Model Profile                    | Value         | Unit    |
|----------------------------------|---------------|---------|
| Selected GPUs                    | None Detected | GPU IDs |
| No. of FLOPs                     | 0.0           | BFLOPs  |
| GPU Memory Requirement           | 0.0062        | GB      |
| Model Parameters                 | 0.0408        | Million |
| Memory Required by Model Weights | 0.1557        | MB      |


2025-04-03 11:35:21.723535: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-03 11:35:21.724926: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [11]:
# Fine-tune model
clustered_model.fit(
  train_images,
  train_labels,
  batch_size=500,
  epochs=1,
  validation_split=0.1)

108/108 [==============================] - 3s 19ms/step - loss: 0.0901 - accuracy: 0.9685 - val_loss: 0.0983 - val_accuracy: 0.9698


In [12]:
_, clustered_model_accuracy = clustered_model.evaluate(
  test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Clustered test accuracy:', clustered_model_accuracy)

Baseline test accuracy: 0.9818000197410583
Clustered test accuracy: 0.9664000272750854


In [13]:
for w in clustered_model.weights:
    print(w.name, w.numpy())

conv2d/bias:0 [-0.00880162 -0.16140667 -0.00986281 -0.14170563 -0.5053878  -0.29706922
 -0.42732382 -0.00310488  0.2989068  -0.26112655 -0.05702841 -0.19512722]
conv2d/kernel:0 [[[[ 0.6414186   0.4315604  -0.83917344  0.07032441  0.06383375
    -0.46468544  0.37808985  0.81573415 -0.04537372  0.2775053
    -1.0989255   0.15231964]]

  [[ 0.91263103  0.26947066 -0.19430073 -0.70849437  0.23527032
     0.3681798   0.3481814   0.52590793  0.31527635  0.29899448
    -0.8427733   0.13663164]]

  [[ 0.9289561  -1.0877162   0.07902098 -0.9345826   0.3765564
     0.42265028 -0.07603849 -1.0387789  -0.77800494 -0.4469227
    -0.14842184 -0.09797344]]]


 [[[-0.29818738  0.79128516 -0.51785403  0.79342765  0.0726191
     0.38634083 -0.0849975   1.0829166  -0.13798949 -0.30349055
    -0.61363727  0.3291322 ]]

  [[ 0.39976525  0.1917658   0.01268764  0.05215013  0.14208715
     0.406272   -0.14213848  0.17550245 -0.55062205  0.42207795
     0.6344701   0.49010646]]

  [[ 0.5656159  -1.1639744  -0

In [14]:
final_model = tfmot.clustering.keras.strip_clustering(clustered_model)

# _, clustered_keras_file = tempfile.mkstemp('.h5')
clustered_keras_file = "/src/final.h5"
print('Saving clustered model to: ', clustered_keras_file)
keras.models.save_model(final_model, clustered_keras_file, 
                           include_optimizer=False)

Saving clustered model to:  /src/final.h5


/tmp/ipykernel_40/217805229.py:6: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  keras.models.save_model(final_model, clustered_keras_file,


In [10]:
for w in final_model.weights:
    print(w.name, w.numpy())

kernel:0 [[[[ 0.44195125 -0.44239596 -1.1691003   0.05833105  0.05833105
     0.05833105  0.44195125 -0.44239596 -0.44239596  0.05833105
    -0.44239596 -0.24850494]]

  [[ 0.44195125 -0.81556934 -0.24850494  0.05833105  0.05833105
     0.05833105  0.6861731   0.23193341 -1.1691003   0.6861731
     0.23193341  0.05833105]]

  [[-1.1691003  -1.1691003   0.6861731   0.23193341 -0.24850494
     0.23193341  0.6861731   0.23193341 -0.44239596  0.6861731
     0.23193341  0.44195125]]]


 [[[ 0.44195125  0.6861731  -1.1691003   0.23193341  0.44195125
     0.23193341  0.44195125  0.05833105 -0.81556934 -0.44239596
     0.23193341  0.05833105]]

  [[ 0.44195125  0.05833105  0.44195125  0.23193341  0.44195125
     0.05833105  0.44195125  0.44195125  0.05833105  0.23193341
     0.23193341  0.23193341]]

  [[-0.81556934 -0.44239596  0.6861731   0.05833105  0.23193341
    -0.24850494 -0.44239596 -0.24850494  0.6861731   0.44195125
     0.05833105  0.05833105]]]


 [[[ 0.23193341  0.23193341 -0.4423

In [11]:
print(model_profiler(final_model, 128))

| Model Profile                    | Value         | Unit    |
|----------------------------------|---------------|---------|
| Selected GPUs                    | None Detected | GPU IDs |
| No. of FLOPs                     | 0.0           | BFLOPs  |
| GPU Memory Requirement           | 0.0062        | GB      |
| Model Parameters                 | 0.0204        | Million |
| Memory Required by Model Weights | 0.0779        | MB      |


2025-04-02 17:17:12.361598: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-02 17:17:12.367716: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [33]:
cluster_centroids = tf.linspace(-1.0, 1.0, 4)

In [34]:
cluster_centroids

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([-1.        , -0.3333333 ,  0.33333337,  1.        ], dtype=float32)>

In [12]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [13]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered Keras model: %.2f bytes" % (get_gzipped_model_size(clustered_keras_file)))

Size of gzipped baseline Keras model: 78217.00 bytes
Size of gzipped clustered Keras model: 12581.00 bytes
